# Assignment 2: Language Modelling in Hangman

Student Name: Hongzhi Fu

Student ID: 1058170

## General info

<b>Due date</b>: Tuesday, 6 April 2021 5pm

<b>Submission method</b>: Canvas submission

<b>Submission materials</b>: completed copy of this iPython notebook

<b>Late submissions</b>: -10% per day (both week and weekend days counted)

<b>Marks</b>: 8% of mark for class (with 7% on correctness + 1% on quality and efficiency of your code)

<b>Materials</b>: See [Using Jupyter Notebook and Python page](https://canvas.lms.unimelb.edu.au/courses/121115/pages/using-jupyter-notebook-and-python?module_item_id=2681264) on Canvas (under Modules>Resources) for information on the basic setup required for this class, including an iPython notebook viewer and the python packages NLTK, Numpy, Scipy, Matplotlib, Scikit-Learn, and Gensim. We recommend installing all the data for NLTK, since you will need various parts of it to complete this assignment. You can also use any Python built-in packages, but do not use any other 3rd party packages (the packages listed above are all fine to use); if your iPython notebook doesn't run on the marker's machine, you will lose marks. <b> You should use Python 3</b>.  

To familiarize yourself with NLTK, here is a free online book:  Steven Bird, Ewan Klein, and Edward Loper (2009). <a href=http://nltk.org/book>Natural Language Processing with Python</a>. O'Reilly Media Inc. You may also consult the <a href=https://www.nltk.org/api/nltk.html>NLTK API</a>.

<b>Evaluation</b>: Your iPython notebook should run end-to-end without any errors in a reasonable amount of time, and you must follow all instructions provided below, including specific implementation requirements and instructions for what needs to be printed (please avoid printing output we don't ask for). You should edit the sections below where requested, but leave the rest of the code as is. You should leave the output from running your code in the iPython notebook you submit, to assist with marking. The amount each question is worth is explicitly given. 

You will be marked not only on the correctness of your methods, but also the quality and efficency of your code: in particular, you should be careful to use Python built-in functions and operators when appropriate and pick descriptive variable names that adhere to <a href="https://www.python.org/dev/peps/pep-0008/">Python style requirements</a>. If you think it might be unclear what you are doing, you should comment your code to help the marker make sense of it.

<b>Updates</b>: Any major changes to the assignment will be announced via Canvas. Minor changes and clarifications will be announced on the discussion board; we recommend you check it regularly.

<b>Academic misconduct</b>: For most people, collaboration will form a natural part of the undertaking of this homework, and we encourge you to discuss it in general terms with other students. However, this ultimately is still an individual task, and so reuse of code or other instances of clear influence will be considered cheating. We will be checking submissions for originality and will invoke the University’s <a href="http://academichonesty.unimelb.edu.au/policy.html">Academic Misconduct policy</a> where inappropriate levels of collusion or plagiarism are deemed to have taken place.

# Overview

In this homework, you'll be creating an 'artificial intelligence' player for the classic Hangman word guessing game. You will need to implement several different automatic strategies based on character-level n-gram language models. Your objective is to create an automatic player which makes the fewest mistakes.

# The Hangman Game (7 marks)

**Instructions**: The <a href="https://en.wikipedia.org/wiki/Hangman_(game)">Hangman game</a> is a simple game whereby one person thinks of a word, which they keep secret from their opponent, who tries to guess the word one character at a time. The game ends when the opponent makes more than a fixed number of incorrect guesses, or they figure out the secret word before then (in which case they *win*). 

Here's a simple version of the game. **No implementation is needed.**

In [1]:
def hangman(secret_word, guesser, max_mistakes=8, verbose=True, **guesser_args):
    """
        This function plays the hangman game with the provided guesser and returns the number of incorrect guesses. 
        
        secret_word: a string of lower-case alphabetic characters, i.e., the answer to the game
        guesser: a function which guesses the next character at each stage in the game
            The function takes a:
                mask: what is known of the word, as a string with _ denoting an unknown character
                guessed: the set of characters which already been guessed in the game
                guesser_args: additional (optional) keyword arguments, i.e., name=value
        max_mistakes: limit on length of game, in terms of number of allowed mistakes
        verbose: silent or verbose diagnostic prints
        guesser_args: keyword arguments to pass directly to the guesser function
    """
    secret_word = secret_word.lower()
    mask = ['_'] * len(secret_word)
    guessed = set()
    if verbose:
        print("Starting hangman game. Target is", ' '.join(mask), 'length', len(secret_word))
    
    mistakes = 0
    while mistakes < max_mistakes:
        if verbose:
            print("You have", (max_mistakes-mistakes), "attempts remaining.")
        guess = guesser(mask, guessed, **guesser_args)

        if verbose:
            print('Guess is', guess)
        if guess in guessed:
            if verbose:
                print('Already guessed this before.')
            mistakes += 1
        else:
            guessed.add(guess)
            if guess in secret_word and len(guess) == 1:
                for i, c in enumerate(secret_word):
                    if c == guess:
                        mask[i] = c
                if verbose:
                    print('Good guess:', ' '.join(mask))
            else:
                if len(guess) != 1:
                    print('Please guess with only 1 character.')
                if verbose:
                    print('Sorry, try again.')
                mistakes += 1
                
        if '_' not in mask:
            if verbose:
                print('Congratulations, you won.')
            return mistakes
        
    if verbose:
        print('Out of guesses. The word was', secret_word)    
    return mistakes

Here is a human guesser allowing interactive play.

In [2]:
def human(mask, guessed, **kwargs):
    """
    This is a simple function for manual play.
    """
    print('\nEnter your guess:')
    return input().lower().strip()

If you want to play hangman interactively, please set `interactive` to `True`. When submitting your solution, set to `False` so we can automatically run the whole notebook using `Run All`.

In [3]:
interactive = False

<b>For your testing:</b>

You can play the game interactively using the following command:

In [4]:
if interactive:
    hangman('whatever', human, 8, True)

### Question 1 (1.0 mark)

**Instructions**: We will use the words in NLTK's Brown corpus for training an artificial intelligence guessing algorithm, and for evaluating the quality of the algorithm.

Your first task is to compute the number of **unique word types** occurring in the Brown corpus, using `nltk.corpus.brown` and the `words` method, and select only words that are **entirely comprised of alphabetic characters**. You should also **lowercase the words**. Finally, randomly shuffle (`numpy.random.shuffle`) this collection of word types, and split them into disjoint training and testing sets. The test set should contain 1000 word types, and the rest should be in the  training set. Note that we are intentionally making the hangman game hard, as the AI will need to cope with test words that it has not seen before, hence it will need to learn generalisable patterns of characters to make reasonable predictions.

**Task**: Collect all unique word types from the Brown corpus, and produce `training_set` and `test_set`, 2 lists that contain 2 disjointed sets of words. Both `training_set` and `test_set` should be a python `list` (as initialised in the code). `test_set` must contain exactly 1000 word types.

**Check**: Use the assertion statements in <b>"For your testing"</b> below for the expected output.

In [5]:
from nltk.corpus import brown
import numpy as np

np.random.seed(1)

# training_set stores the rest word types for training
training_set = []
# test_set stores 1000 word types for testing
test_set = []


###
# Your answer BEGINS HERE
###

brown_words = brown.words()
words = [word.lower() for word in brown_words]
words = set(words)
unique_words = []
for word in words:
    for c in word:
        if c < 'a' or c > 'z':
            break
    else:
        unique_words.append(word)

np.random.shuffle(unique_words)
training_set = unique_words[:-1000]
test_set = unique_words[-1000:]

###
# Your answer ENDS HERE
###

print("Number of word types in test =", len(test_set))
print("Number of word types in train =", len(training_set))

Number of word types in test = 1000
Number of word types in train = 39234


<b>For your testing:</b>

In [6]:
assert(len(training_set) > 35000 and len(training_set) < 45000)
assert(len(test_set) == 1000)

**Play the game**:

Let's see how good you are at this game! Try to guess a random word from the test set. It is surprisingly difficult (and addictive)! Don't forget to set `interactive = True`.

In [7]:
#play hangman using random words from test set
if interactive:
    hangman(np.random.choice(test_set), human, 8, True)

### Question 2 (1.0 mark)

**Instructions**: To set a baseline, your first AI attempt will be a trivial **random method**. For this you should implement a guessing method, similar to the `human` method above, i.e., using the same input arguments and returning a character. Your method should randomly choose a character from the range `a ... z` after excluding the characters that have already been guessed in the current game (all subsequent AI approaches should also exclude previous guesses).

To help you measure the performance of this (and later) guesser, a `test_guesser` method that takes a guesser and measures the average number of incorrect guesses made over all the words in the `test_set` is provided to you. 

**Task**: Complete the `random_guesser` method. It should return a random character from the English alphabets.

**Check**: Use the assertion statements in <b>"For your testing"</b> below for the expected output.

In [8]:
def test_guesser(guesser, test=test_set):
    """
        This function takes a guesser and measures the average number of incorrect guesses made over all the words in the test_set. 
    """
    total = 0
    for word in test:
        total += hangman(word, guesser, 26, False)
    return total / float(len(test))

In [9]:
def random_guesser(mask, guessed, **kwargs):
    
    ###
    # Your answer BEGINS HERE
    ###
    
    alphabet = [chr(i) for i in range(97, 123)]
    while True:
        guessed_c = np.random.choice(alphabet)
        if guessed_c not in guessed:
            return guessed_c
    
    ###
    # Your answer ENDS HERE
    ###

random_word = np.random.choice(test_set)
print("Guessing word =", random_word)
print("Number of mistakes made by the random guesser =", hangman(random_word, random_guesser, 26, False))

result = test_guesser(random_guesser)
print("\nTesting the random guesser using every word in test set")
print("Average number of incorrect guesses: ", result)

Guessing word = continents
Number of mistakes made by the random guesser = 19

Testing the random guesser using every word in test set
Average number of incorrect guesses:  16.87


<b>For your testing:</b>

In [10]:
assert(result > 10 and result < 20)

### Question 3 (1.0 mark)

**Instructions:** As your first real AI, you should train a **unigram language model** over the training set. This requires you to find the frequencies of characters over all training words. Using this model, you should write a guesser that returns the character with the highest probability. Remember to exclude already guessed characters. 

**Task**: Collect the frequencies of characters and store them in `unigram_counts` (use the first answer space). Complete the `unigram_guesser` method. Note that it takes `unigram_counts` as an additional argument (use the second answer space).

**Check**: Use the assertion statements in <b>"For your testing"</b> below for the expected output.

In [11]:
unigram_counts = {}

###
# Your answer BEGINS HERE
###

for word in training_set:
    for c in word:
        unigram_counts[c] = unigram_counts.get(c, 0) + 1

###
# Your answer ENDS HERE
###

def unigram_guesser(mask, guessed, unigram_counts=unigram_counts):

    ###
    # Your answer BEGINS HERE
    ###
    sorted_unigram = sorted(unigram_counts.items(), key=lambda x: x[1], reverse=True)
    
    i = 0
    while True:
        guessed_c = sorted_unigram[i][0]
        if guessed_c not in guessed:
            return guessed_c
        i += 1
    ###
    # Your answer ENDS HERE
    ###


result = test_guesser(unigram_guesser)
print("Testing the unigram guesser using every word in test set")
print("Average number of incorrect guesses: ", result)

Testing the unigram guesser using every word in test set
Average number of incorrect guesses:  10.316


<b>For your testing:</b>

In [12]:
assert(result > 5 and result < 15)

### Question 4 (1.0 mark)

**Instructions:** The length of the secret word is an important clue that we might exploit. Different lengths tend to have different distributions over characters, e.g., short words are less likely to have suffixes or prefixes. You should incorporate this idea by conditioning the unigram model on the length of the secret word, i.e.,  having a **different unigram model for each length**. You will need to be a little careful at test time, to be robust to the situation that you encounter a word length that you didn't see in training. In such a case, your method should behave like the previous `unigram_guesser` in Question 3 (i.e., it guesses characters based on unigram frequencies, unconditioned by the word length).

**Task**: Collect the frequencies of characters conditioned on the word length and store them in `unigram_counts_by_length` (use the first answer space). Complete the `unigram_length_guesser` method (use the second answer space).

**Check**: Use the assertion statements in <b>"For your testing"</b> below for the expected output.

In [13]:
unigram_counts_by_length = {}

###
# Your answer BEGINS HERE
###

for word in training_set:
    length = len(word)
    if length not in unigram_counts_by_length:
        unigram_counts_by_length[length] = {chr(i):0 for i in range(97, 123)}
    for c in word:
        unigram_counts_by_length[length][c] += 1

###
# Your answer ENDS HERE
###

def unigram_length_guesser(mask, guessed, unigram_counts_by_length=unigram_counts_by_length, unigram_counts=unigram_counts):
    ###
    # Your answer BEGINS HERE
    ###
    length = len(mask)
    if length not in unigram_counts_by_length.keys():
        return unigram_guesser(mask, guessed, unigram_counts)
    else:
        unigram_counts_len = unigram_counts_by_length[length]
        sorted_unigram = sorted(unigram_counts_len.items(), key=lambda x: x[1], reverse=True)
        i = 0
        while True:
            guessed_c = sorted_unigram[i][0]
            if guessed_c not in guessed:
                return guessed_c
            i += 1
    ###
    # Your answer ENDS HERE
    ###

result = test_guesser(unigram_length_guesser)
print("Testing the length-conditioned unigram guesser using every word in test set")
print("Average number of incorrect guesses: ", result)

Testing the length-conditioned unigram guesser using every word in test set
Average number of incorrect guesses:  10.22


<b>For your testing:</b>

In [14]:
assert(result > 5 and result < 15)

### Question 5 (1.0 mark)

**Instructions:** Now for the next challenge, you'll build a **bigram language model** over characters. The order of characters is obviously important, yet this wasn't incorporated in any of the above models. Knowing that the word has the sequence `n _ s s` is a pretty strong clue that the missing character might be `e`. Similarly the distribution over characters that start or end a word are highly biased (e.g., toward common prefixes and suffixes, like *un-*, *-ed* and *-ly*).

The task here is to develop a bigram language model over characters, and train it over the training words. Remember to be careful when handling the start of each word properly, e.g., by padding with a special starting symbol such as `$`. Do we also need a special ending symbol? That's for you to decide.

Your bigram guesser should apply your language model to each blank position in the secret word by using its left context character. For example, in the partial word `e _ c _ b _ _` we know the left context for the first three blanks, but have no known left context for the last blank. In the case for the last blank, you should revert to using a unigram language model (since there's no context for us to use the bigram model). You should sum up the probability distribution (over all alphabets from <i>a</i> to <i>z</i>) for the 4 blanks, and select the alphabet with the highest probability that hasn't been guessed.

**Task**: Collect frequency counts that are necessary for building a bigram language model and store them in bigram_counts; feel free to add new objects if needed (use the first answer space). Complete the `bigram_guesser` method (use the second answer space). Note that the method currently only has one additional argument (`bigram_counts`), but you are free to add additional arguments.

In [15]:
import numpy as np
bigram_counts = {}

###
# Your answer BEGINS HERE
###

alphabet = [chr(i) for i in range(97, 123)]
for pre in ['$'] + alphabet:
    bigram_counts[pre] = np.ones(26)  # add 1 for smoothing


for word in training_set:
    word = '$' + word
    for i in range(len(word)-1):
        pre = word[i]; post = word[i+1]
        bigram_counts[pre][ord(post) - ord('a')] += 1
        
# get probability distribution over each bigram
for pre, post in bigram_counts.items():
    post_distribution = bigram_counts[pre] / np.sum(bigram_counts[pre])
    bigram_counts[pre] = post_distribution

# get probability distribution over each unigram
unigram_sum = sum(unigram_counts.values())
unigram_prob = np.zeros(26)
for c, freq in unigram_counts.items():
    idx = ord(c) - ord('a')
    unigram_prob[idx] = freq / unigram_sum

###
# Your answer ENDS HERE
###
    

def bigram_guesser(mask, guessed, bigram_counts=bigram_counts, unigram_counts=unigram_prob): # add extra arguments if needed
    ###
    # Your answer BEGINS HERE
    ###
    sum_prob = np.zeros(26)
    mask_copy = mask.copy()
    mask_copy.insert(0, '$')
    
    for i in range(1, len(mask_copy)):
        if mask_copy[i] == '_':
            if mask_copy[i-1] == '_':
                # use unigram
                sum_prob += unigram_counts
            else:
                # use bigram
                sum_prob += bigram_counts[mask_copy[i-1]]
    
    best_indices = np.argsort(-sum_prob)
    i = 0
    while True:
        best_idx = best_indices[i]
        best_ch = chr(97 + best_idx)
        if best_ch not in guessed:
            return best_ch
        i += 1
        
    ###
    # Your answer ENDS HERE
    ###
    

result = test_guesser(bigram_guesser)
print("Testing the bigram guesser using every word in test set")
print("Average number of incorrect guesses: ", result)

Testing the bigram guesser using every word in test set
Average number of incorrect guesses:  8.662


### Question 6 (1.5 mark)

**Instructions:** You should try to develop a more effective AI for hangman. Feel free to engage your creativity here! Possibilities include better conditioning on the length of the word, fancier smoothing methods, ngram models and bidirectional models (lecture 8). Have fun!

You will be marked based on the performance of your AI model, using a pre-made training and test set (created using a secret seed). Let x be the average number of mistakes in the test set, you will score:
* 1.5 mark if x < 8.0
* 1.0 mark if 8.0 <= x < 8.5
* 0.5 mark if 8.5 <= x < 8.8
* 0.0 mark if x >= 8.8

Note: when testing your AI model's performance, you may want to consider trying different training/test splits (using different seeds) to have a better understanding of its performance, as there will be some variance to its performance depending on the training/test split.

**Task** Complete the `my_amazing_ai_guesser` method, which implements a better language model for hangman.

In [16]:
###
# Your answer BEGINS HERE
###

unigram_prob = np.zeros(26)  # probability distribution over all characters
bigram_prob_forward = {}  # probability distribution of forward bigram model. e.g. p(w_i|w_i-1)
bigram_prob_backward = {}  # probability distribution of backward bigram model e.g. p(w_i|w_i+1)
trigram_prob_forward = {}  # probability distribution of forward trigram model e.g. p(w_i|w_i-1, w_i-2)
trigram_prob_backward = {}  # probability distribution of backward trigram model e.g. p(w_i|w_i+1, w_i+2)
trigram_prob_between = {}  # probability distribution of in-between trigram model e.g. p(w_i|w_i-1, w_i+1)

# frequency of unigram model
for word in training_set:
    for c in word:
        unigram_prob[ord(c) - ord('a')] += 1

# initialize bigram model
alphabet = [chr(i) for i in range(97, 123)]
for pre in ['$'] + alphabet:
    bigram_prob_forward[pre] = np.zeros(26)
    bigram_prob_backward[pre] = np.zeros(26)

# initialize trigram model
for pre1 in ['$'] + alphabet:
    if pre1 == '$':
        for pre2 in ['$'] + alphabet:
            trigram_prob_forward[pre1+pre2] = np.zeros(26)
            trigram_prob_backward[pre1+pre2] = np.zeros(26)
    else:
        for pre2 in alphabet:
            trigram_prob_forward[pre1+pre2] = np.zeros(26)
            trigram_prob_backward[pre1+pre2] = np.zeros(26)
            
for pre in ['$'] + alphabet:
    for post in ['$'] + alphabet:
        trigram_prob_between[pre+post] = np.zeros(26)

# get the frequency of forward bigram model
for word in training_set:
    word = '$' + word
    for i in range(len(word)-1):
        pre = word[i]; post = word[i+1]
        bigram_prob_forward[pre][ord(post) - ord('a')] += 1

# get the frequency of backward bigram model
for word in training_set:
    word = word + '$'
    for i in range(len(word)-1, 0, -1):
        post = word[i]; pre = word[i-1]
        bigram_prob_backward[post][ord(pre) - ord('a')] += 1

# get the frequency of forward trigram model
for word in training_set:
    word = '$$' + word
    for i in range(len(word)-2):
        pre = word[i:i+2]; post = word[i+2]
        trigram_prob_forward[pre][ord(post) - ord('a')] += 1
        
# get the frequency of backward trigram model
for word in training_set:
    word = word + '$$'
    for i in range(len(word)-1, 1, -1):
        post = word[i-1:i+1]; pre = word[i-2]
        trigram_prob_backward[post[::-1]][ord(pre) - ord('a')] += 1

# get the frequency of in-between trigram model
for word in training_set:
    word = '$' + word + '$'
    for i in range(1, len(word)-1):
        cur = word[i]; pre = word[i-1]; post = word[i+1]
        trigram_prob_between[pre+post][ord(cur) - ord('a')] += 1

# get probability distribution over each unigram
unigram_prob = unigram_prob / np.sum(unigram_prob)
    
# get probability distribution over each bigram for both forward and backward
for pre, post_freq in bigram_prob_forward.items():
    post_distribution = (1 + post_freq) / (np.sum(post_freq) + 26)
    bigram_prob_forward[pre] = post_distribution

for post, pre_freq in bigram_prob_backward.items():
    pre_distribution = (1 + pre_freq) / (np.sum(pre_freq) + 26)
    bigram_prob_backward[post] = pre_distribution

# get probability distribution over each trigram for both forward, backward and in-between
for pre, post_freq in trigram_prob_forward.items():
    post_distribution = (1 + post_freq) / (np.sum(post_freq) + 26)
    trigram_prob_forward[pre] = post_distribution

for post, pre_freq in trigram_prob_backward.items():
    pre_distribution = (1 + pre_freq) / (np.sum(pre_freq) + 26)
    trigram_prob_backward[post] = pre_distribution 
    
for prepost, cur_freq in trigram_prob_between.items():
    cur_distribution = (1 + cur_freq) / (np.sum(cur_freq) + 26)
    trigram_prob_between[prepost] = cur_distribution 

def my_amazing_ai_guesser(mask, guessed, 
                          unigram_prob=unigram_prob, 
                          bigram_forward=bigram_prob_forward,
                          bigram_backward=bigram_prob_backward,
                          trigram_forward=trigram_prob_forward,
                          trigram_backward=trigram_prob_backward,
                          trigram_between=trigram_prob_between):
    
    sum_prob = np.zeros(26)  # weighted sum of probability of all models
    
    # padding at the start and end
    mask_copy = mask.copy()
    mask_copy = ['$', '$'] + mask_copy
    mask_copy = mask_copy + ['$', '$']
    
    unigram_weight = 1
    bigram_weight = 1
    trigram_weight = 1
    
    for i in range(2, len(mask_copy)-2):
        cur = mask_copy[i]  # current character
        pre1 = mask_copy[i-1]; pre2 = mask_copy[i-2]  # previous two characters
        post1 = mask_copy[i+1]; post2 = mask_copy[i+2]  # next two characters
        
        # only the unknown character is considered
        if cur == '_':
            # both previous and next characters are known: in-between trigram model
            if pre1 != '_' and post1 != '_':
                sum_prob += trigram_between[pre1+post1] * trigram_weight
            else:
                if pre1 != '_':
                    if pre2 != '_':
                        # previous two characters are known: forward trigram model
                        sum_prob += trigram_forward[pre2+pre1] * trigram_weight
                    else:
                        # only the previous one character is known: forward bigram model
                        sum_prob += bigram_forward[pre1] * bigram_weight
                elif post1 != '_':
                    if post2 != '_':
                        # next two characters are known: backward trigram model
                        sum_prob += trigram_backward[post2+post1] * trigram_weight
                    else:
                        # only the next one character is known: backward bigram model
                        sum_prob += bigram_backward[post1] * bigram_weight
                else:
                    # neither previous nor next character is known: unigram model
                    sum_prob += unigram_prob * unigram_weight
    
    # sort index by non-ascending order
    best_indices = np.argsort(-sum_prob)
    i = 0
    # repeatitively choose the best guessed character
    while True:
        best_idx = best_indices[i]
        best_ch = chr(97 + best_idx)
        if best_ch not in guessed:
            return best_ch
        i += 1

###
# Your answer ENDS HERE
###

result = test_guesser(my_amazing_ai_guesser)
print("Testing my amazing AI guesser using every word in test set")
print("Average number of incorrect guesses: ", result)

Testing my amazing AI guesser using every word in test set
Average number of incorrect guesses:  7.107


### Question 7 (0.5 mark)

**Instructions:** Explain your approach and discuss your result below. Please keep your explanation to a short paragraph.

##### Your answer BEGINS HERE

The approach we use is interpolation, which means combining different order n-gram models. The unigram model simply counts the frequency of each character, and then normalizes to get the probability distribution. The bigram model has two versions: forward and backward. The forward bigram model predicts the probability of occurence of the current character given the previous character, i.e. $p(w_i|w_{i-1})$, while the backward bigram model predicts the probability of occurence of the current character given the next character, i.e. $p(w_i|w_{i+1})$. For robustness, we use Laplace smoothing to handle zero probability. There are three trigram models. The first one is forward trigram, which predicts the probability of current character given the previous two consecutive characters, i.e. $p(w_i|w_{i-1}, w_{i-2})$. The second one is backward trigram, which predicts the probability of current character given the next two consecutive characters, i.e. $p(w_i|w_{i+1}, w_{i+2})$. The last one is in-between trigram, which makes prediction based on the previous one character and the next character, i.e. $p(w_i|w_{i-1}, w_{i+1})$. Similar to bigram, we also introduce Laplace smoothing to deal with zero probability. The final step is guess a word based on the models. For each case, we add a weight term to the probability distribution over all alphabetic letter, then the weighted sum of probability distribution is used to choose the best guessed character.

##### Your answer ENDS HERE